# Garmin Connect scraper

**Garmin Connect** offers a map of user-submitted routes which we can easily filter. Those routes can be accessed by searching for the location (town name). Let's try downloading the *gpx* files for some routes to see if **Garmin** has implemented scraping protection on its website.

In [124]:
#Importing libraries.

import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [125]:
#Initializing our webdriver (Chrome).

driver = webdriver.Chrome()

In [126]:
#Accessing Garmin Connect. At this point we will have to login manually.

driver.get('https://connect.garmin.com/modern/')

In [127]:
#Accessing the activity map.

driver.get('https://connect.garmin.com/modern/courses')

At this stage I'm setting the activity filters manually, since we're only testing the viability of large-scale *gpx* scraping.

In [13]:
#Clicking on the search box.

searchbox = driver.find_element_by_xpath('//*[@id="pageContainer"]/div/div[2]/div[1]/div[1]/input')
searchbox.click()

In [14]:
#Typing a town name and hitting enter.

searchbox.send_keys('Mataró')
time.sleep(0.5)
searchbox.send_keys(Keys.ENTER)

In [15]:
#Accessing the routes displayed on the current map.

routes = driver.find_elements_by_class_name('course-link')

In [16]:
#Accessing a single route.

routes[10].get_attribute('href')

'https://connect.garmin.com/modern/course/15465472'

In [17]:
#Storing all ride links on a list.

route_list = []

for i in routes:
    link = i.get_attribute('href')
    route_list.append(link)

In [19]:
#Inspecting one element of the list.

route_list[0]

'https://connect.garmin.com/modern/course/53159249'

In [20]:
#Let's open a single route link.

driver.get(route_list[0])

In [22]:
#Clicking on the dot button to show the download options.

options = driver.find_element_by_xpath('//*[@id="main-card"]/div/div[4]/div[1]/div/button')
options.click()

In [23]:
#Locating the download button anc clicking. Success!

download = driver.find_element_by_xpath('//*[@id="main-card"]/div/div[4]/div[1]/div/ul/li[2]/a')
download.click()

## Creating a function to download all routes in our list

Now that we've demonstrated the viability of downloading a *gpx* file, let's try to download all routes from our list until we hit a limit.

In [25]:
#We'll begin by creating a loop that performs the desired operation.

start = time.time()

def gpx_downloader(link):
    try:
        driver.get(link) #Accessing the route.
        cond = False
        while cond == False:
            try:
                options = driver.find_element_by_xpath('//*[@id="main-card"]/div/div[4]/div[1]/div/button')
                time.sleep(0.2)
                cond = True
                options.click()
            except:
                cond = False
                time.sleep(0.3)
        cond = False
        while cond == False:
            try:
                download = driver.find_element_by_xpath('//*[@id="main-card"]/div/div[4]/div[1]/div/ul/li[2]/a')
                time.sleep(0.2)
                cond = True
                download.click()
            except:
                cond = False
                time.sleep(0.3)
    except:
        time.sleep(5)
        gpx_downloader(link)
            
for i in route_list:
    gpx_downloader(i)
    
stop = time.time() 
duration = (stop - start)
print('Seconds:', int(duration))

Seconds: 208


In [27]:
#Packing the loop into a function that we can use on lists of links.

def downloader(link_list):
    start = time.time()

    def gpx_downloader(link):
        try:
            driver.get(link) #Accessing the route.
            cond = False
            while cond == False:
                try:
                    options = driver.find_element_by_xpath('//*[@id="main-card"]/div/div[4]/div[1]/div/button')
                    time.sleep(0.2)
                    cond = True
                    options.click()
                except:
                    cond = False
                    time.sleep(0.3)
            cond = False
            while cond == False:
                try:
                    download = driver.find_element_by_xpath('//*[@id="main-card"]/div/div[4]/div[1]/div/ul/li[2]/a')
                    time.sleep(0.2)
                    cond = True
                    download.click()
                except:
                    cond = False
                    time.sleep(0.3)
        except:
            time.sleep(5)
            gpx_downloader(link)

    for i in link_list:
        gpx_downloader(i)

    stop = time.time() 
    duration = (stop - start)
    return print('Seconds:', int(duration))

In [28]:
#Testing the function.

downloader(route_list)

Seconds: 267


## Creating a function that grabs all route links from a destination

Now that we know how to download all *gpx* files from a list of route links we need a way to obtain all route links of a given destination (town).

### Locating elements to navigate the map

In [29]:
#Increase zoom:

more_zoom = driver.find_element_by_xpath('//*[@id="pageContainer"]/div/div[2]/div[2]/a[1]')

In [ ]:
#Increase zoom:

more_zoom = driver.find_element_by_xpath('//*[@id="pageContainer"]/div/div[2]/div[2]/a[1]')

In [45]:
searchbox = driver.find_element_by_xpath('//*[@id="pageContainer"]/div/div[2]/div[1]/div[1]/input')
searchbox.click()
searchbox.send_keys('Mataró')
time.sleep(0.5)
searchbox.send_keys(Keys.ENTER)
time.sleep(0.5)
more_zoom = driver.find_element_by_xpath('//*[@id="pageContainer"]/div/div[2]/div[2]/a[1]')
more_zoom.click()
time.sleep(0.3)
more_zoom.click()

In [46]:
more_zoom = driver.find_element_by_xpath('//*[@id="pageContainer"]/div/div[2]/div[2]/a[1]')
more_zoom.click()
time.sleep(0.3)
more_zoom.click()

In [60]:
routes = driver.find_elements_by_class_name('course-link')

route_list = []

for i in routes:
    link = i.get_attribute('href')
    route_list.append(link)

In [260]:
#These little functions will navigate the Garmin map for us. They can be stacked as needed.

mapa = driver.find_element_by_xpath('//*[@id="leafletMap_396"]')

def down():
    for i in range(10):
        time.sleep(0.4)
        mapa.send_keys(Keys.ARROW_DOWN)
    
def up():
    for i in range(10):
        time.sleep(0.4)
        mapa.send_keys(Keys.ARROW_UP)
    
def left():
    for i in range(21):
        time.sleep(0.4)
        mapa.send_keys(Keys.ARROW_LEFT)
    
def right():
    for i in range(21):
        time.sleep(0.4)
        mapa.send_keys(Keys.ARROW_RIGHT)

In [232]:
#This function appends the displayed routes to the main list.

def getroutes():
    time.sleep(2.5)
    routes = driver.find_elements_by_class_name('course-link')
    for i in routes:
        link = i.get_attribute('href')
        route_list.append(link)

In [270]:
#Second variation, 7x13 grid.

def short():
    start = time.time()

    left()
    left()
    left()
    up()
    up()
    up()
    up()
    up()
    up()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    right()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    right()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    right()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    right()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    right()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    right()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()

    stop = time.time() 
    duration = (stop - start) / 60
    return print('Minutes:', int(duration))

In [269]:
#Third variation, 9x17 grid.

def long():
    start = time.time()

    left()
    left()
    left()
    left()
    up()
    up()
    up()
    up()
    up()
    up()
    up()
    up()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    right()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    right()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    right()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    right()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    right()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    right()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    right()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    up()
    getroutes()
    right()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()
    getroutes()
    down()

    stop = time.time() 
    duration = (stop - start) / 60
    return print('Minutes:', int(duration))

In [271]:
short()

Minutes: 11


In [324]:
len(route_list)

12969

In [322]:
#Creating a dictionary with the route links and saving it.

dict_links = {'links': route_list}
df = pd.DataFrame(dict_links)
df.to_csv('links.csv', index=False)

In [284]:
towns.head()

,nombre,poblacion
4367,Madrid,3334730
880,Barcelona,1664182
7234,València,800215
6162,Sevilla,691395
8124,Zaragoza,681877


In [331]:
list_final[50:]

['Murcia', 'Málaga', 'Zaragoza', 'Sevilla', 'València', 'Barcelona', 'Madrid']

In [ ]:
for i in list_final[50:]:
    searchbox = driver.find_element_by_xpath('//*[@id="pageContainer"]/div/div[2]/div[1]/div[1]/input')
    searchbox.click()
    time.sleep(0.5)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    searchbox.send_keys(Keys.DELETE)
    name = i + ', España'
    searchbox.send_keys(name)
    time.sleep(0.5)
    searchbox.send_keys(Keys.ENTER)
    time.sleep(4)
    long()

cifyamuydu@biyac.com<br>
<br>
123456aA